### Instalación de dependencias

In [1]:
import sys
import subprocess
import os
import warnings
import time
from importlib.util import find_spec
from importlib.metadata import version, PackageNotFoundError

PACKAGES_TO_INSTALL = [ 
                        'langchain==1.2.0',
                        'langchain-google-genai==4.1.1',
                        'langchain-community==0.4.1',
                        'langchain-core==1.2.2',
                        'langsmith==0.5.0',
                        'langgraph==1.0.5',
                        'streamlit==1.52.1'
                      ]
                       
def manage_installation():
    required_restart = False

    # --- Configuración del nivel de logs
    warnings.filterwarnings("ignore")
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    
    print("🔍 Verificando librerías...")
    
    for package in PACKAGES_TO_INSTALL:
        pkg_name = package.split("==")[0]
        try:
            v = version(pkg_name)
            print(f"   ✅ {pkg_name} ya instalado (v{v})")
        except PackageNotFoundError:
            print(f"   ⚠️ {pkg_name} NO encontrado. Instalando...")
            subprocess.check_call([sys.executable, 
                                   "-m", "pip", "install", 
                                   "--quiet", "--no-deps", "--disable-pip-version-check", "--no-warn-script-location",
                                   package])
            required_restart = True
            
    if required_restart:
        print("\n" + "="*50)
        print("🔄 Librerías instaladas. Reiniciando Kernel...")
        print("🛑 Cuando el Kernel se reactive, ejecuta de nuevo esta casilla.")
        print("="*50)
        time.sleep(4) 
        os._exit(00)
    else:
        print("\nEl entorno está configurado correctamente.")

manage_installation()

🔍 Verificando librerías...
   ✅ langchain ya instalado (v1.2.0)
   ✅ langchain-google-genai ya instalado (v4.1.1)
   ✅ langchain-community ya instalado (v0.4.1)
   ✅ langchain-core ya instalado (v1.2.7)
   ✅ langsmith ya instalado (v0.5.0)
   ✅ langgraph ya instalado (v1.0.5)
   ✅ streamlit ya instalado (v1.52.1)

El entorno está configurado correctamente.


### Activación de las API necesarias

In [2]:
%%bash
gcloud services enable apikeys.googleapis.com
gcloud services enable generativelanguage.googleapis.com
gcloud services list --enabled | grep -e 'apikeys' -e 'generative'

apikeys.googleapis.com                    API Keys API
generativelanguage.googleapis.com         Generative Language API


### Definición de variables

In [3]:
# --- Creación y Asignación de las variables
PROJECT_ID = !gcloud config get-value project 2> /dev/null
PROJECT_ID = PROJECT_ID[0].strip()

REGION = "us-central1"
ZONE = "us-central1-a"

INSTANCE_NAME = !gcloud compute instances list \
                   --filter='tags.items:("notebook-instance" "deeplearning-vm")' \
                   --format="value(name)"
INSTANCE_NAME = INSTANCE_NAME[0]

INSTANCE_NAME_IP = !gcloud compute instances describe $INSTANCE_NAME --zone=$ZONE --format='get(networkInterfaces[0].accessConfigs[0].natIP)'
INSTANCE_NAME_IP = INSTANCE_NAME_IP[0].strip()

MODEL_NAME = "gemini-2.5-flash-lite" 

# --- Registro como variables de entorno
os.environ["PROJECT_ID"] = f"{PROJECT_ID}"
os.environ["REGION"] = REGION  
os.environ["ZONE"] = ZONE  
os.environ["INSTANCE_NAME"] = INSTANCE_NAME  
os.environ["INSTANCE_NAME_IP"] = INSTANCE_NAME_IP  
os.environ["MODEL_NAME"] = f"{MODEL_NAME}"

# --- Visualización de las variables 
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")
print(f"ZONE: {ZONE}")
print(f"INSTANCE_NAME: {INSTANCE_NAME}")
print(f"INSTANCE_NAME_IP: {INSTANCE_NAME_IP}")
print(f"MODEL_NAME: {MODEL_NAME}")

PROJECT_ID: qwiklabs-gcp-03-cd2e31c60bb7
REGION: us-central1
ZONE: us-central1-a
INSTANCE_NAME: instance-20260120-150635
INSTANCE_NAME_IP: 34.58.78.71
MODEL_NAME: gemini-2.5-flash-lite


### Creación de una regla de firewall

In [4]:
%%bash
gcloud compute firewall-rules create allow-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8501 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-streamlit' 2> /dev/null

NAME             NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-streamlit  default  INGRESS    1000      tcp:8501        False


### Creación de la credencial API_KEY

In [5]:
import os

GOOGLE_API_KEY=""
API_KEY_NAME="api-key-gemini"

API_KEY_UID = !gcloud services api-keys list --filter="displayName=$API_KEY_NAME" --format="value(name)" --quiet 2> /dev/null
if len(API_KEY_UID) == 0:
    API_KEY = !gcloud services api-keys create --display-name=$API_KEY_NAME --format="value(response.keyString)" --quiet 2> /dev/null
    print(f"API_KEY ({API_KEY_NAME}) creada")  
else:
    print(f"API_KEY ({API_KEY_NAME}) ya existe")
    API_KEY_UID = API_KEY_UID[0].strip()
    API_KEY = !gcloud services api-keys get-key-string $API_KEY_UID | cut -d" " -f2
API_KEY = API_KEY[0].strip()
if API_KEY != "": 
    os.environ["GOOGLE_API_KEY"] = API_KEY  

API_KEY (api-key-gemini) ya existe


### Construcción de la aplicación Python

In [9]:
%%writefile streamlit_swimming.py
import streamlit as st
import os
import warnings
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

# ----------------------------------
#  Configuración
# ----------------------------------

# --- Configuración de la página
st.set_page_config(page_title="Coach Natación AI", page_icon="🏊", layout="centered")

# --- Configuración del nivel de logs
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# --- Lectura de la API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    st.error("❌ Error: Falta la API Key")
    st.stop()

# --- Configuración de variables de entorno
PROJECT_ID = os.environ.get("PROJECT_ID")
MODEL_NAME = os.environ.get("MODEL_NAME")

# ----------------------------------
#  Definición de Funciones (tool)
# ----------------------------------

# --- Función Python que simula buscar en un PDF
@tool
def consultar_minimas_rfen(piscina: str = "50m"):
    """
    ÚSALA SIEMPRE que pregunten por tiempos mínimos, marcas mínimas, mínimas nacionales
    o requisitos de clasificación para campeonatos de España.
    
    Args:
        piscina: El tamaño de la piscina, por defecto '50m' (larga) o '25m' (corta).
    
    Devuelve un JSON simulando la extracción de datos de un PDF oficial de la RFEN.
    """
    time.sleep(2) # Simulamos el tiempo de lectura/procesamiento del PDF

    # Datos simulados (mock) que devolvería el lector de PDF
    datos_simulados = {
        "fuente": "Normativa_RFEN_2024_2025.pdf",
        "piscina": piscina,
        "tiempos_minimos": {
            "50 Libres": "25.10 (M) / 28.50 (F)",
            "100 Libres": "54.80 (M) / 1:01.20 (F)",
            "100 Mariposa": "59.90 (M) / 1:06.50 (F)",
            "200 Estilos": "2:15.50 (M) / 2:28.00 (F)"
        },
        "nota": "Marcas válidas para el Campeonato de España de Verano"
    }
    return datos_simulados

tools = [consultar_minimas_rfen]

# ----------------------------------
# Configuración e Inicialización
# ----------------------------------

# --- System Instructions (Contexto de Natación)
prompt_template = """
    Eres un Asistente Experto en Natación Competitiva y entrenador de alto rendimiento.

    - Si te preguntan por tiempos, mínimas, marcas de clasificación o datos oficiales de la RFEN, DEBES usar la herramienta `consultar_minimas_rfen`.
    - NO inventes tiempos mínimos. Si la herramienta no devuelve el dato exacto, di que son datos aproximados basados en la normativa vigente.
    - Tras usar la herramienta, responde SIEMPRE con este formato claro:
      
      📋 **MÍNIMAS NACIONALES**: 
      - Prueba: [Nombre prueba] -> [Tiempo Masculino] / [Tiempo Femenino]
      *(Añade tantos como hayas recuperado)*
      ⚠️ *Fuente: Normativa RFEN.*

    - Si realizan preguntas técnicas sobre natación (técnica de viraje, cómo mejorar la patada de braza, frecuencia de brazada, zonas de entrenamiento...), responde usando tu conocimiento general como entrenador experto.
    - Si la pregunta NO está relacionada con la natación o el deporte (ej: política, cocina), responde: 'Solo estoy programado para responder sobre natación competitiva'.
"""

# ----------------------------------
#  Configuración del Agente
# ----------------------------------
@st.cache_resource
def iniciar_agente():
    try:
        llm = ChatGoogleGenerativeAI(
            model=MODEL_NAME,
            google_api_key=GOOGLE_API_KEY,
            temperature=0,
            convert_system_message_to_human=True 
        )
        return create_react_agent(llm, tools=tools)
    except Exception as e:
        # Si falla, no podemos hacer mucho más que mostrar error
        return None
     
estado = f"🏊 Conectado a {PROJECT_ID} ({MODEL_NAME}) - Coach AI"     
agent = iniciar_agente()

# ----------------------------------
# Diseño de la interfaz de usuario
# ----------------------------------
st.title("🏊 Chatbot de Natación RFEN")
st.caption(estado)

if not agent:
    st.error("Error al iniciar el agente. Verifica tus variables de entorno.")
    st.stop()

if "messages" not in st.session_state:
    st.session_state.messages = []

# Renderizar historial previo
for msg in st.session_state.messages:
    if isinstance(msg, (HumanMessage, AIMessage)):
        role = "user" if isinstance(msg, HumanMessage) else "assistant"
        with st.chat_message(role):
            st.markdown(msg.content)

# -- Input de Usuario
if prompt := st.chat_input("Ej: ¿Cuál es la mínima de 100 libres Infantil?"):
   
  # -- Escritura del mensaje de usuario
  st.session_state.messages.append(HumanMessage(content=prompt))
  with st.chat_message("user"):
    st.markdown(prompt)

  # -- Lógica del Agente (Asistente)
  with st.chat_message("assistant"):
     
    # -- Variables para guardar el resultado y usarlo fuera
    response = None
    texto_final = ""

    # ---------------------------------------------------------
    # Visualización (Bloque de Pensamiento)
    # ---------------------------------------------------------
    with st.status("⏱️ Analizando tiempos y técnica...", expanded=True) as status:
      try:
        # -- Preparación del input (prompt de entrada)
        sys_msg = SystemMessage(content=prompt_template)
        mensajes_input = [sys_msg] + st.session_state.messages
         
        # -- Ejecución del Agente
        response = agent.invoke({"messages": mensajes_input})
         
        # -- Visualización de pasos intermedios
        mensajes_totales = response["messages"]
        nuevos_mensajes = mensajes_totales[len(mensajes_input):]
        herramienta_usada = False

        for msg in nuevos_mensajes:
          if isinstance(msg, AIMessage) and msg.tool_calls:
            for call in msg.tool_calls:
              st.write(f"📄 **Acción:** Buscando en PDF normativa para `{call['args']}`")
              herramienta_usada = True
          elif isinstance(msg, ToolMessage):
            st.write("💾 **Datos extraídos del PDF:**")
            st.code(msg.content, language="json")

        # Cierre de la caja de Análisis (Pensamiento)
        if herramienta_usada:
          status.update(label="✅ Normativa localizada", state="complete", expanded=False)
        else:
          status.update(label="ℹ️ Respuesta técnica (sin búsqueda)", state="complete", expanded=False)

      except Exception as e:
        status.update(label="❌ Error", state="error")
        st.error(f"Error: {e}")
        st.stop()

    # ---------------------------------------------------------
    #  Respuesta Final
    # ---------------------------------------------------------
    if response:
      ultimo_mensaje = response["messages"][-1]
      raw_content = ultimo_mensaje.content
       
      # --- Limpieza básica
      if isinstance(raw_content, list):
         texto_final = "".join([x.get("text", "") for x in raw_content if isinstance(x, dict)])
      else:
         texto_final = str(raw_content)
       
      # -- Impresión en el Chat Principal
      st.markdown(texto_final)
       
      # -- Almacenamiento en el historial
      st.session_state.messages.append(AIMessage(content=texto_final))

Overwriting streamlit_swimming.py


### Reinicio de Streamlit

In [10]:
!pkill -f streamlit

import time, subprocess

time.sleep(2) 
print(f"🔄 Reiniciando Streamlit con {MODEL_NAME}")
print("\n$ streamlit run streamlit_swimming.py --server.port 8501 > streamlit.log 2>&1 &\n")

entorno = os.environ.copy()

p = subprocess.Popen(
    ["streamlit", "run", "streamlit_swimming.py", "--server.port", "8501"],
    env=entorno,
    stdout=open("streamlit_swimming.log", "w"),
    stderr=subprocess.STDOUT
)

print(f"✅ Listo. Streamlit ejecutándose en segundo plano con PID {p.pid}") 

🔄 Reiniciando Streamlit con gemini-2.5-flash-lite

$ streamlit run streamlit_swimming.py --server.port 8501 > streamlit.log 2>&1 &

✅ Listo. Streamlit ejecutándose en segundo plano con PID 14683


### URL de la aplicación

In [8]:
%%bash 
sleep 5
cat streamlit_langchain.log




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.128.0.2:8501
  External URL: http://34.58.78.71:8501

  Stopping...
